In [ ]:
from pathlib import Path

from pytket import OpType
from pytket.passes import (
    CliffordSimp,
    CommuteThroughMultis,
    EulerAngleReduction,
    FullPeepholeOptimise,
    KAKDecomposition,
    RemoveRedundancies,
    RepeatPass,
    SequencePass,
)
from qiskit.circuit.equivalence_library import SessionEquivalenceLibrary
from qiskit.transpiler.passes import BasisTranslator
from qiskit.transpiler.passes.optimization import (
    CommutationAnalysis,
    CommutativeCancellation,
    HoareOptimizer,
    Optimize1qGatesDecomposition,
    Optimize1qGatesSimpleCommutation,
    TemplateOptimization,
)

from mqt.problemsolver.resource_estimation.fault_tolerant_optimization import (
    SINGLE_QUBIT_AND_CX_QISKIT_STDGATES,
    generate_data_qiskit,
    generate_data_tket,
)

# Qiskit

In [ ]:
benchmarks_dir = Path("mqt_bench")

benchmarks = [f.name.replace(".qasm", "") for f in benchmarks_dir.iterdir() if f.name.endswith(".qasm")]
benchmarks.sort()

basis_gates = SINGLE_QUBIT_AND_CX_QISKIT_STDGATES
transpiler_passes = [
    [
        Optimize1qGatesDecomposition(basis=basis_gates),
        BasisTranslator(SessionEquivalenceLibrary, basis_gates),
    ],
    [
        Optimize1qGatesSimpleCommutation(basis=basis_gates),
        BasisTranslator(SessionEquivalenceLibrary, basis_gates),
    ],
    [
        CommutationAnalysis(),
        CommutativeCancellation(basis_gates=basis_gates),
        BasisTranslator(SessionEquivalenceLibrary, basis_gates),
    ],
    [
        HoareOptimizer(),
        BasisTranslator(SessionEquivalenceLibrary, basis_gates),
    ],
    [
        TemplateOptimization(),
        BasisTranslator(SessionEquivalenceLibrary, basis_gates),
    ],
]

In [ ]:
generate_data_qiskit(
    "results_qiskit.xlsx",
    benchmarks,
    benchmarks_dir,
    transpiler_passes,
)

# Tket

In [ ]:
benchmarks_dir = Path("mqt_bench")

benchmarks = [f.name.replace(".qasm", "") for f in benchmarks_dir.iterdir() if f.name.endswith(".qasm")]
benchmarks.sort()

transpiler_passes = [
    RemoveRedundancies(),
    CliffordSimp(),
    FullPeepholeOptimise(),
    EulerAngleReduction(OpType.Rx, OpType.Ry),
    KAKDecomposition(),
    RepeatPass(SequencePass([CommuteThroughMultis(), RemoveRedundancies()])),
]

transpiler_pass_names = [
    "RemoveRedundancies",
    "CliffordSimp",
    "FullPeepholeOptimise",
    "EulerAngleReduction",
    "KAKDecomposition",
    "RepeatPass",
]

In [ ]:
generate_data_tket(
    "results_tket.xlsx",
    benchmarks,
    benchmarks_dir,
    transpiler_passes,
    transpiler_pass_names,
)